# Informations

To use this script, you have to start with demultiplexed data containing RC primers (TileSeq in my case). The WT sequence has to be the same for every sample.
In theory, you don't have to change the directories if you keep the same folder names.
It's important to have a good sample_sheet.xlsx to start with, it will make the analysis much easier especially if you keep the same headers than me.
I made the script so you don't have to change anything exept in the set up section.

## This is the part 1 of the analysis. Variants count is done in part 2.

### Analysis pipeline : 
### 1- Quality control with fastQC on .fastq.gz files.
### 2- From demultiplexed .fastq.gz files, pandaseq will merge the forward and reverse reads.
### 3- From .fasta files, vsearch will remove the remaining primers.
### 4- From .fasta files, vsearch will aggregate the reads. 
### 5- From .fasta files, needle will align your reads to the wt sequence.
### 6- From .needle files, you will count the number of variants for every codons.

## I'm running everything on Ubuntu wsl for windows downloaded from the appstore. I downloaded Anaconda. It can also be used with the server.

### 1- FastQc = https://anaconda.org/bioconda/fastqc
### 2- PandaSeq = https://anaconda.org/bioconda/pandaseq
### 3- Vsearch = https://anaconda.org/bioconda/vsearch
### 4- Needle from Emboss = https://anaconda.org/bioconda/emboss

## Quality controls are done along the way with graphs and dataframes to know the quantity of reads we are dealing with.

# Import

In [ ]:
import gzip
import Bio
import subprocess
import openpyxl
from Bio import SeqIO
import os,sys,re

import pandas as pd
print(pd.__name__, pd.__version__)

import numpy as np
print(np.__name__, np.__version__)

import matplotlib.pyplot as plt
import matplotlib
print(matplotlib.__name__, matplotlib.__version__)
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import MaxNLocator

import scipy.stats as stats
import scipy
print(scipy.__name__, scipy.__version__)

import seaborn as sns
print(sns.__name__, sns.__version__)

from collections import Counter

# Set up
Ideally this is the only cell which has to be modified.

In [ ]:
##see the sample sheet to know how to fill it
##info = dataframe of the sample_sheet
## wt = the path to the wt sequence in a .fa file
## wt_seq = wt sequence directly in a string
## *the wt sequence has to be without the primers, so how it will be once trimmed.

if not os.path.exists('CaERG11-F4'):
    os.makedirs('CaERG11-F4')

if not os.path.exists('CaERG11-F4-merged_reads'):
    os.makedirs('CaERG11-F4-merged_reads')
    
if not os.path.exists('CaERG11-F4-figures'):
    os.makedirs('CaERG11-F4-figures')

if not os.path.exists('CaERG11-F4-Trimmed'):
    os.makedirs('CaERG11-F4-Trimmed')

if not os.path.exists('CaERG11-F4-Aggregated'):
     os.makedirs('CaERG11-F4-Aggregated')

if not os.path.exists('CaERG11-F4-variants'):
    os.makedirs('CaERG11-F4-variants')
    
if not os.path.exists('CaERG11-F4-ReadsCount'):
    os.makedirs('CaERG11-F4-ReadsCount')
    
if not os.path.exists('CaERG11-F4-Needle'):
    os.makedirs('CaERG11-F4-Needle')
    
if not os.path.exists('CaERG11-F4-Heatmaps'):
    os.makedirs('CaERG11-F4-Heatmaps')


date = "2023-06-20" #date your doing the analysis, used to save figures
experiment = "CaERG11-F4" #generic name of your experiment, used to save figures
reads_folder = "./CaERG11-F4/" #location of the reads in .gz
merged_folder = "./CaERG11-F4-merged_reads/" # location of merged reads
figures_folder = "./CaERG11-F4-figures/" # location of the figures
trim_folder = "./CaERG11-F4-Trimmed/" #location of trimmed reads
agg_folder = "./CaERG11-F4-Aggregated/" #location of aggregated reads
needle_folder = "./CaERG11-F4-Needle/" #location of reads aligned to wt (.needle)
variants_folder = "./CaERG11-F4-variants/" #location of the variants count
reads_counts_folder = "./CaERG11-F4-ReadsCount/" #location of all dataframes output in excel format with reads counts 

sample_sheet = "./sample_sheet_F4_all.xlsx" #excel sheet with all the information

pe = 250 #paired-end

before_nut = 1 #number of nucleotides before the first complete codon of your sequence (0, 1 or 2)
after_nut = 1 #number of nucleotides after the last complete codon of your sequence (0, 1 or 2)

aa_start = 393 #first amino acid of your sequence (complete codon)
aa_end = 514 #last amino acid of your sequence (complete codon)

wt = "./CaERG11_wt_seq/CaERG11_F4_wt_seq.fasta" #your WT sequence AFTER trimming
wt_seq = "ATATTCAGAAAAGTTACTAACCCACTTAGGATCCCTGAAACCAACTACATCGTCCCAAAAGGACACTACGTTCTTGTCAGCCCAGGCTACGCACACACGAGTGAGAGATACTTTGATAACCCGGAGGATTTTGATCCTACACGTTGGGATACTGCTGCAGCCAAAGCCAATTCTGTAAGCTTTAACTCCAGTGATGAGGTAGATTACGGCTTTGGGAAAGTATCAAAAGGCGTCAGCTCACCATATCTTCCCTTCGGTGGCGGTAGACATAGATGTATAGGTGAACAATTTGCATACGTTCAGCTGGGAACCATATTAACGACGTTTGTTTATAACTTGAGATGGACTATCGACGGGTATAAAGTCCCTGATCCTGACTATAGCTCTATGGTTGTTctaCCCACCGAA"
pos_mutated = [401,402,403,404,405,406,407,408,410,432,434,436,446,447,448,449,450,452,456,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,479,480,481,483,484,503,504,505,506,507,508,509,510,511]
codons_mutated = ["GCT","GCC","TGC","TGT","GAT","GAC","GAG","GAA",
"TTC","TTT","GGT","GGA","CAC","CAT","ATC","ATT","AAG","AAA","TTG","TTA","ATG","AAC","AAT","CCT","CCA","CAG","CAA",
"AGA","CGT","TCT","TCC","ACC","ACT","GTT","GTC","TGG","TAC","TAT","TAA"] 
codons_all = ["GCT","GCC","TGC","TGT","GAT","GAC","GAG","GAA",
"TTC","TTT","GGT","GGA","CAC","CAT","ATC","ATT","AAG","AAA","TTG","TTA","ATG","AAC","AAT","CCT","CCA","CAG","CAA",
"AGA","CGT","TCT","TCC","ACC","ACT","GTT","GTC","TGG","TAC","TAT","TAA","GTA","GCA","GTG","ATA","GCG","CTC","CTA","CGG",
"TAG","TGA","CCC","GGG","TCG","AGG","CGA","CGC","AGT","CTG","ACG","TCA","AGC","GGC","CTT","ACA","CCG"]
wt_len = len(wt_seq)
wt_aa="IFRKVTNPLRIPETNYIVPKGHYVLVSPGYAHTSERYFDNPEDFDPTRWDTAAAKANSVSFNSSDEVDYGFGKVSKGVSSPYLPFGGGRHRCIGEQFAYVQLGTILTTFVYNLRWTIDGYKVPDPDYSSMVVLPTE"


In [ ]:
info = pd.read_excel(sample_sheet, header=0, index_col=0)
info

# FastQC

In [ ]:
#Define a funtion to get the FastQC

def quality_fastqc(input_fastq_file):

    subprocess.check_output('fastqc '+input_fastq_file, shell=True)
    # Run Fastqc on the compressed sequencing file

    return

In [ ]:
%%capture
#don't print the output
##Let's iterate trough all the sequencing file and use our quality_fastqc function
 
# giving file extension we want
ext = ('fastq.gz')
 
# iterating over all files
#for files in os.listdir(reads_folder):
    #if files.endswith(ext):
        #quality_fastqc(reads_folder + files)

In [ ]:
##find the number of reads in the fastq.gz files
#we divide by 4 because there is 4 lines in the file for 1 read


before_merge_depth_dict = {}

for Sample in list(info.index):
    fullpath = reads_folder +info.loc[Sample]['reads_file_For']
    print(fullpath)
    call ="gunzip -c " + fullpath + " |wc -l"
    result = subprocess.check_output(call, shell=True)
    numseqs = int(result)/4.0
    before_merge_depth_dict[str(info.loc[Sample]["Name"])] = numseqs

In [ ]:
reads_before = pd.DataFrame.from_dict(before_merge_depth_dict, orient="index", columns = ["reads_before"])
reads_before

In [ ]:
# let's look at how much read we have before merging
#fig, ax = plt.subplots(figsize=(25,15))

#ax.set_yscale('log', base=2)

#df_size = len(info.index)

#plt.bar(info["Name"], reads_before["reads_before"])

#plt.title("Reads before merging")
#plt.xlabel('Samples', fontsize=14)
#plt.ylabel('Number of reads in log2', fontsize=14)
#plt.xticks(rotation=90)

#name = figures_folder + "Reads_Before_" + experiment + date  
#plt.savefig(f"{name}.png", format='png', dpi=300,bbox_inches="tight")

# Merge reads with Pandseq

In [ ]:
def merge_reads(Sample):
    # this function generates and runs a Pandaseq call to merge R1 and R2 for each demultiplexed sample.
    # it returns the starting number of reads in the file, which is also stored in a dict

    filepath_for = reads_folder + info.loc[Sample]['reads_file_For']
    filepath_rev = reads_folder + info.loc[Sample]['reads_file_Rev']
    
                
    filepath_out = merged_folder+str(info.loc[Sample]['Name'])+'.fasta'
        # define the merging output filepath
        
    for_len = len(info.loc[Sample]['RC_for_seq'])
    rev_len = len(info.loc[Sample]['RC_rev_seq'])
    wt_len = len(wt_seq)
        
        #size = wt_len+for_len+rev_len
        #maxi = str(size + 50)
        #mini = str(size - 50)
        #overlap = ((size-pe) - pe)*-1
        #over_max = str(overlap +25)
        #over_min = str(overlap -25)
        #size = str(size)        
        #panda_seq_call = 'pandaseq -f '+filepath_for+' -r '+filepath_rev+" -L " + maxi +" -l " + mini +" -O " + over_max +" -o "+ over_min+" -k + 4 -B -N -t 0.5 -T 6 -w "+filepath_out
        #print(panda_seq_call)
        
    print("LOL")
    panda_seq_call = 'pandaseq -f '+filepath_for+' -r '+filepath_rev+' -L 500 -l 350 -O 100 -o 70 -k 4 -B -N -t 0.5 -T 6 -w '+ filepath_out
    subprocess.check_output(panda_seq_call, shell=True)
        # generates and runs a pandseq call with the following parameters:
        #    -L              maximum length of assembled sequence: 550 pb
        #    -O              maximum overlap between reads: 400 pb
        #    -k             number of sequence locations per kmer (increased from default value of 2)
        #    -B                 allow input to lack a barcode
        #    -N                 eliminate all sequences with Ns
        #    -t 0.5             score threshold
        #    -T 6               use 6 threads
        #    -w filepath_out    write output to filepath_out
        
    return filepath_out
    # return initial number of reads

In [ ]:
%%capture
for Sample in list(info.index):
    # for sample in DataFrame
    merge_reads(Sample)
    # merge reads

In [ ]:
after_merge_depth_dict = {}
after_merge_depth_list = []
# empty container that wil hold the post-merge number of reads for each sample

def count_merged(Sample):
    
    filepath = merged_folder
    filepath_merged = str(info.loc[Sample]['Name'])+'.fasta'
    filepath+=filepath_merged
    print(filepath)
    # find merged reads fasta file path based on sample name
    
    after_merge_depth = 0
    # counter for read number
      
    with open(filepath, 'r') as source:
        for line in source:
            if line.startswith('>'):
                after_merge_depth += 1
                # open merged read file, loop through lines,
                # find the headers and increment by one for each header 

    after_merge_depth_dict[str(info.loc[Sample]["Name"])] = after_merge_depth
    after_merge_depth_list.append(after_merge_depth)
    # once file has been read through, add read number to the dictionary
    return after_merge_depth
    # return merged read counts

In [ ]:
for Sample in list(info.index):
    # for each sample
    count_merged(Sample)
    # count the number of reads after merging

In [ ]:
after_merge_depth_dict

In [ ]:
reads_after_merge = pd.DataFrame.from_dict(after_merge_depth_dict, orient="index", columns = ["reads_after_merge"])
reads_after_merge

In [ ]:
# let's look at how much read we got after merging

#fig, ax = plt.subplots(figsize=(25,15))

#ax.set_yscale('log', base=2)

#df_size = len(info.index)
#plt.bar(info["Name"], reads_after["reads_after"])

#plt.title("Reads after merging")
#plt.xlabel('Samples', fontsize=14)
#plt.ylabel('Number of reads in log2', fontsize=14)
#plt.xticks(rotation=90)

#name = figures_folder + "Reads_After_Merge_" + experiment+"_" + date  
#plt.savefig(f"{name}.png", format='png', dpi=300,bbox_inches="tight")

In [ ]:
##supperpose before and after merging number of reads

fig = plt.figure(figsize=(25,15))
ax = fig.add_subplot(111)
ax.bar(x=info["Name"], height=reads_before["reads_before"], width=0.35,align='center')
ax.bar(x=info["Name"], height=reads_after_merge["reads_after_merge"], width=0.35/2,  align='center')
plt.yscale("log", base=2)
plt.title("Number of reads before and after merging")
plt.xticks(rotation=90, size =8)
plt.ylabel('Number of reads in log2', size=12)

name = figures_folder + "Reads_Before_After_Merge_" + experiment+"_" + date  
plt.savefig(f"{name}.png", format='png', dpi=300,bbox_inches="tight")
plt.show()

# Trim and aggregate reads with vsearch

In [ ]:
def trim_aggregate(Sample):
    # This function generates and runs two vsearch calls to trim and then aggregate the merged reads.
    # Trimming removes the variable degenreate sequences, which then  allows us to aggregate the sequences that have
    # the same sequence with minimal influence from amplicons regions used for multiplexing.
       
    
    filepath_merged = merged_folder+str(info.loc[Sample]['Name'])+'.fasta'
    # get the merged read filepath based on the sample
    
    trim_path = trim_folder + str(info.loc[Sample]['Name'])+ '.fasta'
    aggregate_path = agg_folder +str(info.loc[Sample]['Name'])+ '.fasta'
    # generate output filepaths for the trimming and aggregating steps
    
    #we need the length of the primers to cut them
    for_len = str(len(info.loc[Sample]['RC_for_seq'])+before_nut)
    rev_len = str(len(info.loc[Sample]['RC_rev_seq'])+after_nut)
    
    vsearch_trim_call = 'vsearch --fastx_filter '+filepath_merged+" --fastq_stripleft "+ for_len+ " --fastq_stripright "+ rev_len+" --fastaout " + trim_path
    subprocess.check_output(vsearch_trim_call, shell=True)
    # generates and runs a vsearch call with the following parameters:
    #    --fastx_filter filepath_merged    vsearch program to be used and input: fastxfilter can perform different
    #                                      operations on fastq files
    #    --fastq_stripleft                 removes 76 bases from the 5p end
    #    --fastq_stripright 76             removes 76 bases from the 3p end
    #    --fastaout trim_path              ouput trimmed sequences the trim_path file                 
    
    vsearch_aggregate_call = 'vsearch --derep_fulllength '+ trim_path +' --relabel seq --output '+aggregate_path+' --sizeout'
    subprocess.check_output(vsearch_aggregate_call, shell=True)
    # generates and runs a vsearch call with the following parameters:
    #    --derep_fulllength trim_path    vsearch program to be used and input. derep_fulllength aggregates perfectly identical 
    #                                    sequences in the input fasta, dramatically reducing the number of sequences that then 
    #                                    need to be aligned
    #    --relabel seq                   changes header so that all sequence names begin by seq
    #    --output                        path where output will be written
    #    --sizeout                       append the size of the sequence cluster to the fasta header
    


In [ ]:
%%capture

for Sample in list(info.index):
    
    trim_aggregate(Sample)

In [ ]:
###Get unique/different and singleton sequences in aggregated reads

after_agg_unique_dict = {}
after_agg_single_dict = {}
after_agg_two_dict = {}

for Sample in list(info.index):
    #Find the aggregated fasta file
    file_to_count=agg_folder + str(info.loc[Sample]['Name'])+'.fasta'
    #write bash command to count all unique reads per library
    cmdline_unique=f'grep -c ">" {file_to_count}'
    #Run bash line to count unique reads
    unique_seq_number=subprocess.getoutput(cmdline_unique)
    #Write number of unique sequences to dict
    after_agg_unique_dict[str(info.loc[Sample]["Name"])] = int(unique_seq_number)
    #Write bash line to count all sequences with one read (singletons)
    cmdline_singleton = f'grep -c "size=1" {file_to_count}'
    #Run bash line to count singletons
    singleton_number=subprocess.check_output(cmdline_singleton, shell=True)
    #Write number of singleton sequences to dict
    after_agg_single_dict[str(info.loc[Sample]["Name"])] = int(singleton_number)
    #Write bash line to count all sequences with 2 reads (almost singletons)
    cmdline_two = f'grep -c "size=2" {file_to_count}'
    #Run bash line to count singletons
    two_number=subprocess.check_output(cmdline_two, shell=True)
    #Write number of singleton sequences to df
    after_agg_two_dict[str(info.loc[Sample]["Name"])] = int(two_number)

del(file_to_count, cmdline_unique, unique_seq_number, cmdline_singleton, singleton_number, Sample )


In [ ]:
seq_agg = pd.DataFrame.from_dict(after_agg_unique_dict, orient="index", columns = ["nbr_seq_unique"])
seq_agg

In [ ]:
seq_agg_single = pd.DataFrame.from_dict(after_agg_single_dict, orient="index", columns = ["nbr_seq_single"])
seq_agg_single

In [ ]:
seq_agg_two = pd.DataFrame.from_dict(after_agg_two_dict, orient="index", columns = ["nbr_seq_two"])
seq_agg_two

# let's look at how much different reads we got after aggregate

fig, ax = plt.subplots(figsize=(25,15))

ax.set_yscale('log', base=2)

df_size = len(info.index)
plt.bar(info["Name"], seq_agg["nbr_seq_unique"])

plt.title("Different Reads after aggregate")
plt.xlabel('Samples', fontsize=14)
plt.ylabel('Number of reads in log2', fontsize=14)
plt.xticks(rotation=90)

name = figures_folder + "Reads_After_Aggregate_Unique_" + experiment+"_" + date  
plt.savefig(f"{name}.png", format='png', dpi=300,bbox_inches="tight")

# let's look at how much singleton reads we got after aggregate

fig, ax = plt.subplots(figsize=(25,15))

ax.set_yscale('log', base=2)

df_size = len(info.index)
plt.bar(info["Name"], seq_agg_single["nbr_seq_single"])

plt.title("Singleton reads after aggregate")
plt.xlabel('Samples', fontsize=14)
plt.ylabel('Number of reads in log2', fontsize=14)
plt.xticks(rotation=90)

name = figures_folder + "Reads_After_Aggregate_Single_" + experiment +"_"+ date  
plt.savefig(f"{name}.png", format='png', dpi=300,bbox_inches="tight")

In [ ]:
#Let's look at the size distribution



nbr_reads_single = {}

for Sample in list(info.index):
    file_to_count=agg_folder + str(info.loc[Sample]['Name'])+'.fasta'
    
    with open(file_to_count, 'r') as source:
        

        exp = str(info.loc[Sample]['Name'])
        print(exp)
        
        singleton=0
        other=0
        
        for line in source:
            if "size=1" in line:
                singleton += 1
            else :
                other += 1
                
        nbr_reads_single[str(info.loc[Sample]["Name"])] = int(singleton)

        print(str(singleton))
        print(str(other))

        #sns.set(style="white")
        #plt = sns.histplot(data=size, x=size, color = "green", bins = 10, binwidth=0.01)
        #plt.set_yscale("log", base=2)
        #title = str(info.loc[Sample]['Name'])
        #plt.set_xlabel("Occurence of unique aggregated sequence size", fontsize = 10)
        #plt.set_title(title)
        #plt.set_ylabel("Log Count", fontsize = 10)

        #plt.tick_params(axis='x', rotation=90)
        #fig = plt.figure
        #fig
        #name = figures_folder + "Number_size_Aggregated_Log_" + exp +"_"+ date  
        #fig.savefig(f"{name}.png", format='png', dpi=300,bbox_inches="tight")

In [ ]:
nbr_reads_single = pd.DataFrame.from_dict(nbr_reads_single, orient="index", columns = ["nbr_reads_single"])
nbr_reads_single


In [ ]:
##supperpose before and after merging number of reads

fig = plt.figure(figsize=(25,15))
ax = fig.add_subplot(111)
ax.bar(x=info["Name"], height=reads_after_merge["reads_after_merge"], width=0.35,align='center')
ax.bar(x=info["Name"], height=nbr_reads_single["nbr_reads_single"], width=0.35/2,  align='center')
plt.yscale("log", base=2)
plt.title("Total number of reads and singletons",size=12)
plt.xticks(rotation=90, size =8)
plt.ylabel('Number of reads in log2', size=12)

name = figures_folder + "Reads_Total_and_Singletons_" + experiment+"_" + date  
plt.savefig(f"{name}.png", format='png', dpi=300,bbox_inches="tight")
plt.show()

In [ ]:
##Let's do a dataframe with all the reads informations

nbr_reads_dict = {}

for Sample in list(info.index):

    nbr_reads_dict[str(info.loc[Sample]["Name"])] = info.loc[Sample]["nbr_reads"]

nbr_reads_dict
reads_wanted = pd.DataFrame.from_dict(nbr_reads_dict, orient="index", columns = ["reads_wanted"])
df1 = pd.concat([reads_wanted, reads_before], axis=1)
df1['reads_percent'] = (df1['reads_before'] / df1['reads_wanted'])*100
df1 = pd.concat([df1, reads_after_merge], axis=1)
df1['merge_percent'] = (df1['reads_after_merge'] / df1['reads_before'])*100
df1 = pd.concat([df1, seq_agg], axis=1)
df1 = pd.concat([df1, seq_agg_single], axis=1)
df1['nbr_single_seq_percent'] = (df1['nbr_seq_single'] / df1['nbr_seq_unique'])*100

df1 = pd.concat([df1, nbr_reads_single], axis=1)
df1['percent_singletons'] = (df1['nbr_reads_single'] / df1['reads_after_merge'])*100
df1
name = reads_counts_folder + "Reads_count_" + experiment+"_" + date  
df1.to_excel(f"{name}.xlsx")
df1.to_csv(f"{name}.csv") ##save it

# Align to WT sequence with needle

In [ ]:
%%capture

for Sample in list(info.index):
    
    #Define file paths
    file_in= agg_folder +str(info.loc[Sample]['Name'])+ '.fasta'
    file_out= needle_folder +str(info.loc[Sample]['Name'])+ '.needle'
    
    #Write command line to run the needle alignment tool
    needle_call = 'needle -auto -gapopen 100 -asequence '+ wt + ' -bsequence '+ file_in +' -aformat3 markx10 -outfile '+file_out   
    subprocess.check_output(needle_call, shell = True)

    # generate and run a needle call with the following parameters:
    #    -auto                        do not run in interactive mode
    #    -gapopen 50                  increase penalty for gap in the alignment. This helps
    #                                 with instances where there are no indels in the read but
    #                                 needle still opens a gap 
    #    -asequence ref_fasta_path    reference sequence fasta file
    #    -bsequence filepath          query sequences in fasta format
    #    -aformat3 markx10            choose the ouput format
    #    -ourfile needle_out          define output file path